<a href="https://colab.research.google.com/github/dajebbar/Diabetes-Classification/blob/main/Multicollinearity_and_VIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multicollinearity & VIF(Variance Inflation Factor)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
diabetes_df = pd.read_csv('./diabetes.csv')
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
def zero_check(df):
  features_list = df.columns[:-1].tolist()
  for feature in features_list:
    print(f'No of zeros in {feature} : {df[df[feature] <= 0].shape[0]}')

In [4]:
zero_check(diabetes_df)

No of zeros in Pregnancies : 111
No of zeros in Glucose : 5
No of zeros in BloodPressure : 35
No of zeros in SkinThickness : 227
No of zeros in Insulin : 374
No of zeros in BMI : 11
No of zeros in DiabetesPedigreeFunction : 0
No of zeros in Age : 0


In [5]:
def zero_to_mean(df):
  list = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
  for item in list:
    df[item] = np.where(df[item] < 0, 0, df[item])
    print(f'{item} mean: {df[item].mean()}')
    df[item] = df[item].replace(0, df[item].mean())

In [6]:
zero_to_mean(diabetes_df)
print()
zero_check(diabetes_df)

Glucose mean: 120.89453125
BloodPressure mean: 69.10546875
SkinThickness mean: 20.536458333333332
Insulin mean: 79.79947916666667
BMI mean: 31.992578124999977

No of zeros in Pregnancies : 111
No of zeros in Glucose : 0
No of zeros in BloodPressure : 0
No of zeros in SkinThickness : 0
No of zeros in Insulin : 0
No of zeros in BMI : 0
No of zeros in DiabetesPedigreeFunction : 0
No of zeros in Age : 0
